In [ ]:
import numpy as np, matplotlib.pyplot as plt
from pyhnc import *

In [ ]:
N = int(1e5)
Δr = 0.01
grid = Grid(N, Δr)
r, q = grid.r, grid.q

A, rho0 = 25, 3
potential = potentials.DPD(A)
solvent = Solver(grid, alpha=0.1).solve(potential, rho0)

In [ ]:
class DPDWall(planar.Wall):
    def __getstate__(self):
        return {'A': self.A, 'rcut': self.rcut}

    def __init__(self, A, rcut):
        self.A = A
        self.rcut = rcut

    def potential(self, x):
        x = np.asarray(x)
        phi = 0.5*self.A * x**2 / self.rcut
        phi = np.where(x > 0, 0., phi)
        phi = np.where(x <= -self.rcut, 0.5*self.A * self.rcut, phi)
        return phi.item() if np.isscalar(x) else phi

    def force(self, x):
        x = np.asarray(x)
        f = -self.A * x/self.rcut**2
        f = np.where(x > 0, 0., f)
        f = np.where(x <= -self.rcut, 0., f)
        return f.item() if np.isscalar(x) else f

wall = DPDWall(25, 5)
x = np.linspace(-10, 10, 1000)
plt.plot(x, wall.potential(x), label='potential')
plt.plot(x, wall.force(x), label='force')
plt.legend(loc='best')
plt.xlabel('$x$')
plt.show()

In [ ]:
plane = planar.PlanarSolver(solvent, alpha=0.1)
plane.solve(wall, monitor=True, restart=True)

In [ ]:
plt.plot(r, solvent.c, label=r'$c(r)$')
plt.plot(plane.x, plane.cpar, '--', label=r'$c_\parallel(x)$')
plt.legend(loc='best')
plt.xlim([0, 2])
plt.xlabel(r'$r$ or $x$')
plt.ylabel(r'$c$')

plt.figure()
plt.plot(plane.x, plane.density_profile)
plt.xlabel('$x$')
plt.ylabel(r'$\rho(x)$')
plt.xlim([-5, 5])

plt.show()

In [ ]:
with np.load('data/wall_dpd.npz') as f:
    x, samples, box = f['x'], f['rho'], f['box']
    nsamples = samples.shape[-1]
    profile = np.average(samples, axis=0)

A = 25
rho0 = np.average(profile[-10:])
potential = potentials.DPD(A)
solvent = Solver(grid).solve(potential, rho0)

L = box[0]
wall = DPDWall(A, 0.25*L)
plane = planar.PlanarSolver(solvent, alpha=0.1)
plane.solve(wall, monitor=True)

In [ ]:
plt.plot(x, profile, 'o', mfc='None', label='MD')
plt.plot(plane.x, plane.density_profile, 'k--', label='HNC')
plt.legend(loc='best')
plt.xlabel('$x$')
plt.ylabel(r'$\rho(x)$')
plt.xlim([-10, 5])
plt.show()

# Surface tension

In [ ]:
class DPDSemiSoftWall(planar.Wall):
    def __getstate__(self):
        return {'A': self.A}

    def __init__(self, A):
        self.A = A

    def potential(self, x):
        phi = 0.5 * self.A * (1 - x)**2
        phi[x > 1] = 0.
        phi[x < 0] = np.inf
        return phi

    def force(self, x):
        f = self.A * (1 - x)
        f[x > 1] = 0.
        f[x < 0] = 0.
        return f

A, rho0 = 25, 3
potential = potentials.DPD(A)
solvent = Solver(grid).solve(potential, rho0)

plane = planar.PlanarSolver(solvent, alpha=0.1)

Awall_hnc = np.linspace(0, 60., 50)
gamma_hnc = np.empty_like(Awall_hnc)
for i, eps in enumerate(Awall_hnc):
    wall = DPDSemiSoftWall(eps)
    gamma_hnc[i] = plane.solve(wall).surface_tension

In [ ]:
Awall, gamma = np.genfromtxt('data/jimenezSerratos2025_surface_tension.csv').T

plt.figure(figsize=(3.375, 2.1))
pl, = plt.plot(Awall, gamma, 'o', mfc='None',
               label=r'Jimenez-Serratos \emph{et al.} (2025)')
plt.plot(Awall_hnc, gamma_hnc, '-', c=pl.get_color(), label='planar HNC')

plt.legend(loc='best')
plt.xlabel(r'$A_\mathrm{wall}$')
plt.ylabel(r'$\gamma$')
plt.xlim([0, 60])
plt.ylim([-10, 15])

plt.show()